<a href="https://colab.research.google.com/github/ffy208/DriveCost/blob/main/Google_Cloud_x_MLB_Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import vertexai
from google.cloud import aiplatform
from google.cloud.aiplatform import gapic as aiplatform_gapic
from google.cloud import storage
import json
import matplotlib.ticker as ticker
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import seaborn as sns

# EDA

In [ ]:
url = "https://statsapi.mlb.com/api/v1/teams/119/roster?season=2024"
response = requests.get(url)
data = response.json()

for player in data["roster"][:5]:
    print(player["person"]["fullName"], "-", player["position"]["abbreviation"])


In [ ]:
player_url = "https://statsapi.mlb.com/api/v1/people/660271?season=2024"
player_data = requests.get(player_url).json()
print(json.dumps(player_data, indent=2))


In [ ]:
game_url = "https://statsapi.mlb.com/api/v1.1/game/716463/feed/live"
game_data = requests.get(game_url).json()
print(json.dumps(game_data, indent=2))


In [ ]:
url = "https://storage.googleapis.com/gcp-mlb-hackathon-2025/datasets/2024-mlb-homeruns.csv"
df = pd.read_csv(url)

print(df.head())
print(df.info())


In [ ]:
print(df.head())
print(df.info())
print(df.describe())


In [ ]:
print(f"number of payers: {df['play_id'].nunique()}")


In [ ]:
print(df.columns)

In [ ]:
plt.figure(figsize=(8,5))
df["ExitVelocity"].hist(bins=50)
plt.xlabel("Exit Velocity (mph)")
plt.ylabel("Count")
plt.title("Exit Velocity Distribution")
plt.show()


In [ ]:
url = "https://statsapi.mlb.com/api/v1/teams/119/roster?season=2024"
response = requests.get(url).json()

for player in response["roster"]:
    print(f"Player ID: {player['person']['id']} - {player['person']['fullName']}")


In [ ]:
player_id = 660271
url = f"https://statsapi.mlb.com/api/v1/people/{player_id}/stats?stats=season&season=2023"

response = requests.get(url).json()

print(response)


In [ ]:
player_id = 660271  # Shohei Ohtani
url = f"https://statsapi.mlb.com/api/v1/people/{player_id}/stats?stats=yearByYear&group=hitting"

response = requests.get(url).json()
print(response)


In [ ]:
url = "https://statsapi.mlb.com/api/v1/people?season=2024&sportId=1"
response = requests.get(url).json()

print(response)


In [ ]:
url = "https://statsapi.mlb.com/api/v1/teams?season=2024&sportId=1&hydrate=roster"
response = requests.get(url).json()

print(response)


In [ ]:
team_ids = [team["id"] for team in response["teams"]]
print(team_ids)


In [ ]:
player_ids = []

for team_id in team_ids:
    url = f"https://statsapi.mlb.com/api/v1/teams/{team_id}/roster?season=2024"
    response = requests.get(url).json()


    for player in response["roster"]:
        player_ids.append(player["person"]["id"])

print(player_ids[:10])

In [ ]:
player_id = player_ids[0]
url = f"https://statsapi.mlb.com/api/v1/people/{player_id}/stats?stats=yearByYear&season=2023"

response = requests.get(url).json()
print(response)


In [ ]:
teams_url = "https://statsapi.mlb.com/api/v1/teams?sportId=1&season=2024"
teams_response = requests.get(teams_url).json()

team_ids = [team["id"] for team in teams_response["teams"]]
print(f"Total Teams: {len(team_ids)}")
print("First 5 teams:", team_ids[:5])


In [ ]:
player_ids = []

for team_id in team_ids:
    roster_url = f"https://statsapi.mlb.com/api/v1/teams/{team_id}/roster?season=2024"
    roster_response = requests.get(roster_url).json()

    for player in roster_response.get("roster", []):
        player_ids.append(player["person"]["id"])

print(f"Total Players: {len(player_ids)}")
print("First 5 players:", player_ids[:5])


In [ ]:
player_stats = {}

for player_id in player_ids[:10]
    stats_url = f"https://statsapi.mlb.com/api/v1/people/{player_id}/stats?stats=yearByYear&season=2023"
    stats_response = requests.get(stats_url).json()
    player_stats[player_id] = stats_response.get("stats", [])

print("Sample player stats:", list(player_stats.items())[:2])


In [ ]:
parsed_stats = {}

for player_id, stats in player_stats.items():
    for stat_group in stats:
        for split in stat_group.get("splits", []):
            if split.get("season") == "2023":
                parsed_stats[player_id] = split["stat"]


print(json.dumps(list(parsed_stats.items())[:2], indent=4))


In [ ]:
df = pd.DataFrame.from_dict(parsed_stats, orient="index")

display(df)

In [ ]:
print(df_sorted["era"].isna().sum())


In [ ]:
df_sorted = df_sorted.dropna(subset=["era"])

In [ ]:
df_sorted["era"] = pd.to_numeric(df_sorted["era"], errors="coerce")

In [ ]:
plt.figure(figsize=(10,5))
plt.barh(df_sorted.index.astype(str), df_sorted["era"])
plt.xlabel("ERA")
plt.ylabel("Player ID")
plt.show()


In [ ]:


plt.figure(figsize=(10, 5))
plt.barh(df_sorted.index.astype(str), df_sorted["era"])
plt.xlabel("ERA")
plt.ylabel("Player ID")
plt.gca().xaxis.set_major_locator(ticker.MultipleLocator(0.5))
plt.show()


In [ ]:



def get_all_players():
    url = "https://statsapi.mlb.com/api/v1/sports/1/players"
    response = requests.get(url).json()


    player_ids = [player["id"] for player in response["people"]]
    return player_ids

def get_player_stats(player_id, season):
    url = f"https://statsapi.mlb.com/api/v1/people/{player_id}/stats?stats=yearByYear&season={season}"
    response = requests.get(url).json()


    stats = response.get("stats", [])
    if stats:
        for split in stats[0].get("splits", []):
            if split["season"] == str(season):
                return split["stat"]
    return None

def get_historical_data(player_ids, seasons):
    all_data = []
    for player_id in player_ids[:50]:
        for season in seasons:
            stats = get_player_stats(player_id, season)
            if stats:
                stats["player_id"] = player_id
                stats["season"] = season
                all_data.append(stats)

    return pd.DataFrame(all_data)

player_ids = get_all_players()
seasons = list(range(2015, 2024))
df = get_historical_data(player_ids, seasons)

df.to_csv("mlb_historical_data.csv", index=False)

print(df.head())


In [ ]:
print(df.columns)

In [ ]:
print("Columns in df:", df.columns)

In [ ]:
df.info()

In [ ]:

features = ["gamesPlayed", "atBats", "runs", "hits", "homeRuns", "strikeOuts",
            "baseOnBalls", "avg", "obp", "slg", "ops", "stolenBases", "rbi", "iso", "bb_k_ratio"]

df = df[["player_id", "season"] + features]


df = df.fillna(0)
df = df.replace({"": 0})
df[features] = df[features].astype(float)
print(df.info())


In [ ]:
y = df["ops"]
X = df[features]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


y_pred = model.predict(X_test)


mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.4f}")
print(f"R² Score: {r2:.4f}")


In [ ]:

df_2024 = df.copy()
df_2024["season"] = 2024


X_2024 = scaler.transform(df_2024[features])


df_2024["predicted_ops"] = model.predict(X_2024)


print(df_2024[["player_id", "season", "ops", "predicted_ops"]].head(10))


In [ ]:
def get_player_names(player_ids):
    player_name_dict = {}

    for player_id in player_ids:
        url = f"https://statsapi.mlb.com/api/v1/people/{player_id}"
        response = requests.get(url)
        data = response.json()

        if "people" in data and data["people"]:
            player_name_dict[player_id] = data["people"][0]["fullName"]

    return player_name_dict


player_ids = df["player_id"].unique()
player_names = get_player_names(player_ids)


In [ ]:
print(df.columns)


In [ ]:
print(df_2024.head())


In [ ]:
print(df_2024[["player_name", "player_id", "season", "ops", "predicted_ops"]].head(10))


In [ ]:
ALL_PLAYERS_API = "https://statsapi.mlb.com/api/v1/sports/1/players"

response = requests.get(ALL_PLAYERS_API).json()
player_ids = [player["id"] for player in response.get("people", [])]

all_player_stats = []

In [ ]:
print(len(player_ids))
print(player_ids[:10])

In [ ]:
for player_id in player_ids:
    stats_response = requests.get(PLAYER_STATS_API.format(player_id=player_id)).json()
    if "stats" in stats_response:
        for stat_group in stats_response["stats"]:
            for stat in stat_group.get("splits", []):
                if stat["season"] == "2023":
                    player_data = {
                        "player_id": player_id,
                        "player_name": stat["player"]["fullName"],
                        "season": 2023,
                        "gamesPlayed": stat["stat"].get("gamesPlayed", np.nan),
                        "atBats": stat["stat"].get("atBats", np.nan),
                        "runs": stat["stat"].get("runs", np.nan),
                        "hits": stat["stat"].get("hits", np.nan),
                        "homeRuns": stat["stat"].get("homeRuns", np.nan),
                        "strikeOuts": stat["stat"].get("strikeOuts", np.nan),
                        "baseOnBalls": stat["stat"].get("baseOnBalls", np.nan),
                        "avg": stat["stat"].get("avg", np.nan),
                        "obp": stat["stat"].get("obp", np.nan),
                        "slg": stat["stat"].get("slg", np.nan),
                        "ops": stat["stat"].get("ops", np.nan),
                        "stolenBases": stat["stat"].get("stolenBases", np.nan),
                        "rbi": stat["stat"].get("rbi", np.nan)
                    }
                    all_player_stats.append(player_data)


df = pd.DataFrame(all_player_stats)


In [ ]:
df.to_parquet("mlb_2023_data.parquet", index=False)

In [ ]:

df_2023 = pd.read_parquet("mlb_2023_data.parquet")


seasons = list(range(2015, 2023))
all_player_stats = []

for season in seasons:
    for player_id in player_ids:
        stats_response = requests.get(PLAYER_STATS_API.format(player_id=player_id)).json()
        if "stats" in stats_response:
            for stat_group in stats_response["stats"]:
                for stat in stat_group.get("splits", []):
                    if int(stat["season"]) == season:
                        player_data = {
                            "player_id": player_id,
                            "player_name": stat["player"]["fullName"],
                            "season": season,
                            "gamesPlayed": stat["stat"].get("gamesPlayed", np.nan),
                            "ops": stat["stat"].get("ops", np.nan),
                        }
                        all_player_stats.append(player_data)

df_other_seasons = pd.DataFrame(all_player_stats)


df_all = pd.concat([df_2023, df_other_seasons], ignore_index=True)

df_all.to_parquet("mlb_all_seasons.parquet", index=False)


In [ ]:
df_all.info()

In [ ]:

df = pd.read_parquet("mlb_all_seasons.parquet")

float_cols = ["avg", "obp", "slg", "ops"]
for col in float_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")


df.fillna(0, inplace=True)

features = ["gamesPlayed", "atBats", "runs", "hits", "homeRuns", "strikeOuts", "baseOnBalls", "obp", "slg"]
target = "ops"

In [ ]:

X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

In [ ]:

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"MSE: {mse}")
print(f"R²: {r2}")

In [ ]:

df_2024 = df[df["season"] == 2023].copy()
X_2024 = scaler.transform(df_2024[features])
df_2024["predicted_ops"] = model.predict(X_2024)

In [ ]:
print(df_2024[["player_name", "player_id", "season", "ops", "predicted_ops"]].head(10))

In [ ]:
BASE_URL = "https://statsapi.mlb.com/api/v1"

def get_all_players():
    url = f"{BASE_URL}/sports/1/players"
    response = requests.get(url)
    data = response.json()
    players = [
        {"player_id": p["id"], "player_name": p["fullName"]}
        for p in data["people"]
    ]

    return pd.DataFrame(players)

def get_player_stats(player_id, season=2024):
    url = f"{BASE_URL}/people/{player_id}/stats?stats=yearByYear&season={season}"
    response = requests.get(url)
    data = response.json()

    if "stats" in data and len(data["stats"]) > 0:
        stats = data["stats"][0]["splits"]
        if len(stats) > 0:
            stat_data = stats[0]["stat"]
            return {
                "player_id": player_id,
                "season": season,
                "gamesPlayed": stat_data.get("gamesPlayed", 0),
                "atBats": stat_data.get("atBats", 0),
                "runs": stat_data.get("runs", 0),
                "hits": stat_data.get("hits", 0),
                "homeRuns": stat_data.get("homeRuns", 0),
                "strikeOuts": stat_data.get("strikeOuts", 0),
                "baseOnBalls": stat_data.get("baseOnBalls", 0),
                "avg": stat_data.get("avg", 0),
                "obp": stat_data.get("obp", 0),
                "slg": stat_data.get("slg", 0),
                "ops": stat_data.get("ops", 0),
                "stolenBases": stat_data.get("stolenBases", 0),
                "rbi": stat_data.get("rbi", 0),
            }
    return None


def get_2024_season_data():
    all_players_df = get_all_players()

    all_stats = []
    for idx, row in all_players_df.iterrows():
        player_id = row["player_id"]
        player_name = row["player_name"]

        player_stats = get_player_stats(player_id, 2024)
        if player_stats:
            player_stats["player_name"] = player_name
            all_stats.append(player_stats)

        time.sleep(0.5)

    df_2024 = pd.DataFrame(all_stats)
    return df_2024

df_2024 = get_2024_season_data()


df_2024.to_parquet("mlb_2024_season.parquet", index=False)


print(df_2024.head(10))


In [ ]:
df_2024.info()

In [ ]:

df_all = pd.read_parquet("mlb_all_seasons.parquet")


df_all = pd.concat([df_all, df_2024], ignore_index=True)


df_all.to_parquet("mlb_all_seasons_updated.parquet", index=False)


print(df_all.info())
print(df_all.tail(10))


# Data cleansing

In [ ]:
df = pd.read_parquet("mlb_all_seasons_updated.parquet")


float_cols = ["avg", "obp", "slg", "ops"]
for col in float_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")


df.fillna(0, inplace=True)


features = ["gamesPlayed", "atBats", "runs", "hits", "homeRuns",
            "strikeOuts", "baseOnBalls", "stolenBases", "rbi"]
target = "ops"

df_train = df[df["season"] < 2024]
df_test = df[df["season"] == 2024]


X_train, X_val, y_train, y_val = train_test_split(
    df_train[features], df_train[target], test_size=0.2, random_state=42
)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(df_test[features])


# Training a local model

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)


y_pred_rf = rf_model.predict(X_val_scaled)


mse_rf = mean_squared_error(y_val, y_pred_rf)
r2_rf = r2_score(y_val, y_pred_rf)

print(f"Random Forest 误差: MSE={mse_rf}, R²={r2_rf}")


# Training Vertex AI

In [ ]:
from google.cloud import aiplatform


aiplatform.init(project="GCl_MLB_Hackathon", location="us-central1")
bq_source = "bq://GCl_MLB_Hackathon.mlb_dataset.mlb_cleaned"

dataset = aiplatform.TabularDataset.create(
    display_name="mlb-ops-dataset",
    bq_source=bq_source
)

model = aiplatform.AutoMLTabularTrainingJob(
    display_name="mlb-ops-model",
    optimization_prediction_type="regression",
    optimization_objective="minimize-rmse"
)

vertex_model = model.run(
    dataset=dataset,
    target_column="ops",
    budget_milli_node_hours=1000
)

print(f"Vertex AI: {vertex_model.resource_name}")


## Add More Models

In [ ]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(n_estimators=500, max_depth=6, learning_rate=0.05, random_state=42)
xgb_model.fit(X_train_scaled, y_train)


y_pred_xgb = xgb_model.predict(X_val_scaled)

mse_xgb = mean_squared_error(y_val, y_pred_xgb)
r2_xgb = r2_score(y_val, y_pred_xgb)

print(f"XGBoost MSE={mse_xgb}, R²={r2_xgb}")


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(128, activation="relu", input_shape=(X_train_scaled.shape[1],)),
    layers.Dropout(0.2),
    layers.Dense(64, activation="relu"),
    layers.Dense(32, activation="relu"),
    layers.Dense(1)
])


model.compile(optimizer="adam", loss="mse", metrics=["mae"])

history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_data=(X_val_scaled, y_val))


y_pred_nn = model.predict(X_val_scaled).flatten()


mse_nn = mean_squared_error(y_val, y_pred_nn)
r2_nn = r2_score(y_val, y_pred_nn)

print(f"MLP: MSE={mse_nn}, R²={r2_nn}")


### **Summary of MLB Prospect Prediction Model**

#### **Objective**
The goal of this project was to predict a baseball prospect’s potential career impact based on historical and current performance data. The dataset consisted of multiple seasons' worth of Major League Baseball (MLB) player statistics, including traditional and advanced metrics.

---

#### **Data Processing & Feature Engineering**
1. **Data Collection & Storage**  
   - Fetched MLB player data via an API.
   - Saved combined historical and 2024 season data in **Google Cloud Storage (GCS)** as `mlb_all_seasons.csv`.
   - Attempted to load the dataset into **Vertex AI** but encountered permission/authentication issues.
   - Used **pandas** to preprocess the dataset locally.

2. **Feature Engineering**
   - Selected relevant features including `gamesPlayed`, `atBats`, `homeRuns`, `strikeOuts`, `ops`, etc.
   - Created additional derived metrics such as `BB/K Ratio`, `ISO`, and `BABIP`.
   - Standardized numerical features using `StandardScaler`.

---

#### **Modeling Approach**
Three different machine learning models were trained and evaluated:

1. **Random Forest Regressor (Baseline Model)**
   - `n_estimators=100, random_state=42`
   - Provided a **Mean Squared Error (MSE) of ~0.033** with an **R² score of ~0.18**.

2. **XGBoost Regressor**
   - `learning_rate=0.05, max_depth=6, n_estimators=500`
   - Slight improvement over Random Forest but still exhibited relatively low R² scores.

3. **Neural Network (MLP)**
   - A **Multi-Layer Perceptron (MLP)** with 3 fully connected layers.
   - **Trained for 50 epochs** with a batch size of 32.
   - Achieved a **MSE of ~0.0331 and R² of ~0.17**, indicating similar performance to tree-based models.

---

#### **Prediction Results**
- After training, the models were used to **predict the OPS for 2024 season players**.
- **Final predicted OPS scores were generated for all available players**.
- The results were stored in a **pandas DataFrame**, containing:
  - `player_name`
  - `player_id`
  - `season`
  - `actual OPS (if available)`
  - `predicted OPS`

- Example output:
  
  ```
  player_name   player_id  season   ops   predicted_ops
  Andrew Abbott  671096    2024    0.741      0.74033
  CJ Abrams      682928    2024    0.605      0.60510
  Bryan Abreu    650556    2024    0.391      0.38890
  José Abreu     547989    2024    0.849      0.84903
  ```

---

#### **Challenges Encountered**
1. **Vertex AI Integration Issues**
   - Permission errors when creating a dataset on **Google Vertex AI**.
   - Authentication issues with **Google Cloud IAM roles and service accounts**.
   - Ultimately, local machine learning models were used instead of **AutoML**.

2. **Data Processing Issues**
   - Missing and `NaN` values in certain key metrics.
   - Encountered `infinity` values during `StandardScaler` transformation.
   - Addressed by **removing NaN values and imputing missing features**.

3. **Model Performance**
   - **All models struggled with low R² scores (~0.17–0.18)**, indicating that OPS is a difficult target to predict with the available features.
   - Feature selection and additional data sources may be needed for improved predictive accuracy.

---

#### **Next Steps**
- **Improve Model Performance**
  - Consider **deep learning architectures** beyond MLP, such as LSTMs or Transformers.
  - Introduce **time-series analysis** since player performance changes over seasons.
  - Incorporate **player physical metrics (height, weight, injuries, age)**.

- **Enhance Data Quality**
  - Expand features with **external baseball datasets**.
  - Ensure **consistency in season-over-season player tracking**.
  - Add **game-specific data** (e.g., home/away performance, opponent strength).

- **Optimize Google Cloud Integration**
  - Resolve **Vertex AI dataset permissions**.
  - Leverage **BigQuery ML** for large-scale processing.
  - Deploy a real-time prediction API using **Cloud Functions**.

---

### **Final Thoughts**
While the current models provide a **basic prediction for 2024 OPS**, the results suggest room for improvement. Future iterations should focus on **better feature selection, additional data sources, and improved machine learning models** to enhance accuracy. **Google Cloud services** can be better utilized to scale the solution and deploy real-time inference.